# Import the Necessary Libraries

In [1]:
#IMPORT THE PANDAS AND NUMPY LIBRARIES
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2f}'.format
import warnings
warnings.filterwarnings('ignore')

#IMPORT THE SCIKIT-LEARN LIBRARIES
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from mpl_toolkits.mplot3d import Axes3D

#IMPORT THE VISUALIZATION LIBRARIES
import plotly.express as px
import kaleido
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#IMPORT THE SQALCHEMY LIBRARY's CREATE_ENGINE METHOD
import mysql.connector
from mysql.connector import Error

# Import data from MySQL DB

In [2]:
try:
    connection = mysql.connector.connect(host='scraper.cx53soegx3qk.eu-west-1.rds.amazonaws.com',
                                         database='pipedrive',
                                         user='pipedrive',
                                         password='#8LsH25%ZD', use_pure=True)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        df_properties = pd.read_sql('SELECT * FROM ade_properties_intestati', connection)
        df_leads_deals = pd.read_sql('SELECT * FROM leads_deals_person', connection)
        print("Connected to MySQL database... MySQL Server version on ", db_Info)
except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL database... MySQL Server version on  8.0.35
MySQL connection is closed


# Data Preprocessing

### Dropping unnecesary columns

In [11]:
#DROP THE COLUMNS THAT ARE NOT NEEDED
df_properties.drop(['buyer_persona1_1imm','buyer_persona1_2imm','buyer_persona1_more3imm','buyer_persona2_1imm','buyer_persona2_2imm','buyer_persona2_more3imm','buyer_persona3_1imm','buyer_persona3_2imm','buyer_persona3_more3imm','buyer_persona4_1imm','buyer_persona4_2imm','buyer_persona4_more3imm','buyer_persona5_1imm','buyer_persona5_2imm','buyer_persona5_more3imm','buyer_persona6_1imm','buyer_persona6_2imm','buyer_persona6_more3imm','buyer_persona7_1imm','buyer_persona7_2imm','buyer_persona7_more3imm','buyer_persona8_1imm','buyer_persona8_2imm','buyer_persona8_more3imm','ricerca_nazionale_done_at','status_ricerca_nazionale','sent','has_phone','is_lead','qualified_ready_nel_passato','person_add_time',], axis=1, inplace=True)

In [12]:
df_properties

,id_intestati,codfisc,attribution,quota,titolarita,Total_IMM,IMM_MILANO_HINTERLAND_No_small_cities,IMM_MILANO_HINTERLAND_Small_cities,IMM_ROMA_TERRITORIO,IMM_ROMA_PROVINCIA,IMM_TORINO_TERRITORIO,IMM_TORINO_PROVINCIA,IMM_FIRENZE_TERRITORIO,IMM_FIRENZE_PROVINCIA,IMM_GENOVA_TERRITORIO,IMM_GENOVA_PROVINCIA,IMM_OTHER
0,091aac50-96e4-4231-8d48-1e3c5b55a328,-C.L.I.COMPAGNIA_LIGURE_INVESTIMENTI,Milano,None,Proprieta',1,1,1,0,0,0,0,0,0,0,0,0
1,1136dc3a-2321-4c46-ae2f-2167d2826d62,-IMPRESA_SOC.COSTR.TRONI_OMILDE_E_FIGLI-S.N.C.,Firenze,None,,2,0,0,0,0,0,0,2,0,0,0,0
2,c563d10f-d166-4770-8640-208b55a7cd5a,-SOCIETA'_NUOVA_EDILIZIA_S._DESIDERIO,Genova,1000/1000,Proprieta',1,0,0,0,0,0,0,0,0,1,0,0
3,91fe7f41-3f98-480d-aa06-b64c8cdfddaf,._ROMA_.,Roma,1/1,Diritto del concedente concedente,5,0,0,5,0,0,0,0,0,0,0,0
4,f090add1-1e3a-45d9-8596-7b83de972541,''DISCO_PIU'_S.R.L._''_CON_SEDE_IN_MILANO_VIA_...,Milano,None,Proprieta',1,1,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3931469,90eddf85-75fd-4255-8767-3bd3add0bd31,ZZZVTR60S58L219E,Torino,1/2,Proprieta',2,0,0,0,0,2,0,0,0,0,0,0
3931470,690db6fa-15e0-4f08-9728-5d3e0d0c30b7,ZZZVTR76H25A509E,Roma,1/2,Proprieta',1,0,0,1,0,0,0,0,0,0,0,0
3931471,28f41fdb-a57a-4136-8ca2-db0b04372bb5,ZZZVTR94A02H501S,Roma,10/40,Proprieta' per 10/40,1,0,0,0,1,0,0,0,0,0,0,0
3931472,940a0090-038e-4a9e-90a8-3f4b41e3c151,ZZZWLY27H50L273E,Torino,1/1,Proprieta' per 1/1,1,0,0,0,0,0,1,0,0,0,0,0


In [13]:
#DROP THE COLUMNS THAT ARE NOT NEEDED
df_leads_deals.drop(['person_id','property_id','mkt_acquisition_campaign','mkt_acquisition_content','deal_id','phone','lead_id','Qualified_Ready_nel_passato','creator_user_id','pre_sales'], axis=1, inplace=True)

In [14]:
df_leads_deals

,mkt_acquisition_channel,Codice_Fiscale,person_add_time,is_archived,status,city,pipeline_id,is_lead,title,owner_name
0,Outbound,DNGWQN61T10Z210K,2024-01-12 00:31:58,1,deal,Milano - Zibido san Giacomo,0,1,DONG WENQIAN,
1,Outbound,GRSGGR60C06F205K,2023-05-05 14:56:21,0,deal,Milano,0,1,GREGORIO GRASSI,
2,Outbound,PRTPMR59D44L219I,2024-01-11 22:30:17,0,deal,Torino,0,1,PORTINARO PAOLA MARIA,
3,Outbound,RMSLSN71T23H501Z,2023-11-30 12:21:32,1,deal,Roma,0,1,RIMASSA ALESSANDRO,
4,,,2022-11-07 10:07:22,1,deal,Brescia,0,1,26857,
...,...,...,...,...,...,...,...,...,...,...
257357,Outbound,VLSMSM64B24L219H,2024-01-11 22:58:49,2,open,Torino,2,0,VALESANO MASSIMO,Riccardo Del Monaco
257358,SocialPaid,,2024-04-01 16:44:54,2,open,Monza,1,0,34513,Fabio
257359,Outbound,CPDSNT66M65L219Y,2023-10-25 18:20:52,2,open,Torino,2,0,CAPODICI SIMONETTA,Riccardo Del Monaco
257360,Outbound,PZZLSN55A68L219N,2023-12-12 11:52:42,2,open,Milano,2,0,PIZZI ALESSANDRA,Tania Aureli


### Merging the two dataframes in OBT

In [15]:
#MERGE THE TWO DATAFRAMES
df = pd.merge(df_properties, df_leads_deals, left_on='codfisc', right_on='Codice_Fiscale', how='left')

In [16]:
df

,id_intestati,codfisc,attribution,quota,titolarita,Total_IMM,IMM_MILANO_HINTERLAND_No_small_cities,IMM_MILANO_HINTERLAND_Small_cities,IMM_ROMA_TERRITORIO,IMM_ROMA_PROVINCIA,...,mkt_acquisition_channel,Codice_Fiscale,person_add_time,is_archived,status,city,pipeline_id,is_lead,title,owner_name
0,091aac50-96e4-4231-8d48-1e3c5b55a328,-C.L.I.COMPAGNIA_LIGURE_INVESTIMENTI,Milano,None,Proprieta',1,1,1,0,0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1136dc3a-2321-4c46-ae2f-2167d2826d62,-IMPRESA_SOC.COSTR.TRONI_OMILDE_E_FIGLI-S.N.C.,Firenze,None,,2,0,0,0,0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c563d10f-d166-4770-8640-208b55a7cd5a,-SOCIETA'_NUOVA_EDILIZIA_S._DESIDERIO,Genova,1000/1000,Proprieta',1,0,0,0,0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,91fe7f41-3f98-480d-aa06-b64c8cdfddaf,._ROMA_.,Roma,1/1,Diritto del concedente concedente,5,0,0,5,0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,f090add1-1e3a-45d9-8596-7b83de972541,''DISCO_PIU'_S.R.L._''_CON_SEDE_IN_MILANO_VIA_...,Milano,None,Proprieta',1,1,1,0,0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3945940,90eddf85-75fd-4255-8767-3bd3add0bd31,ZZZVTR60S58L219E,Torino,1/2,Proprieta',2,0,0,0,0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3945941,690db6fa-15e0-4f08-9728-5d3e0d0c30b7,ZZZVTR76H25A509E,Roma,1/2,Proprieta',1,0,0,1,0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3945942,28f41fdb-a57a-4136-8ca2-db0b04372bb5,ZZZVTR94A02H501S,Roma,10/40,Proprieta' per 10/40,1,0,0,0,1,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3945943,940a0090-038e-4a9e-90a8-3f4b41e3c151,ZZZWLY27H50L273E,Torino,1/1,Proprieta' per 1/1,1,0,0,0,0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Removing Duplicates

In [49]:
#CHECK THE SHAPE OF THE DATAFRAME
df.shape

(3945057, 27)

In [23]:
#Drop Duplicates Rows
df.drop_duplicates(inplace=True)

### Data Imputation

In [30]:
#CHECK NULL VALUES
df.isnull().sum() / len(df) * 100

id_intestati                             0.00
codfisc                                  0.00
attribution                              0.00
quota                                    6.71
titolarita                               0.11
Total_IMM                                0.00
IMM_MILANO_HINTERLAND_No_small_cities    0.00
IMM_MILANO_HINTERLAND_Small_cities       0.00
IMM_ROMA_TERRITORIO                      0.00
IMM_ROMA_PROVINCIA                       0.00
IMM_TORINO_TERRITORIO                    0.00
IMM_TORINO_PROVINCIA                     0.00
IMM_FIRENZE_TERRITORIO                   0.00
IMM_FIRENZE_PROVINCIA                    0.00
IMM_GENOVA_TERRITORIO                    0.00
IMM_GENOVA_PROVINCIA                     0.00
IMM_OTHER                                0.00
mkt_acquisition_channel                 94.23
Codice_Fiscale                          94.23
person_add_time                         94.23
is_archived                             94.23
status                            

In [48]:
#CHECK THE DATA TYPES
df.dtypes

id_intestati                                     object
codfisc                                          object
attribution                                      object
quota                                            object
titolarita                                       object
Total_IMM                                         int64
IMM_MILANO_HINTERLAND_No_small_cities             int64
IMM_MILANO_HINTERLAND_Small_cities                int64
IMM_ROMA_TERRITORIO                               int64
IMM_ROMA_PROVINCIA                                int64
IMM_TORINO_TERRITORIO                             int64
IMM_TORINO_PROVINCIA                              int64
IMM_FIRENZE_TERRITORIO                            int64
IMM_FIRENZE_PROVINCIA                             int64
IMM_GENOVA_TERRITORIO                             int64
IMM_GENOVA_PROVINCIA                              int64
IMM_OTHER                                         int64
mkt_acquisition_channel                         

In [44]:
#QUOTA/TITOLITARIA: FILL THE NULL VALUES OF QUOTA WITH THE MODE VALUE
df['quota'].fillna(df['quota'].mode()[0], inplace=True)
df['titolarita'].fillna(df['quota'].mode()[0], inplace=True)

In [46]:
#Looking for the most frequent values in the column
df.groupby('quota')['quota'].count().sort_values(ascending=False)

quota
1/1              1534869
1/2              1155452
1000/1000         138743
0.50              119948
1.00              118333
                  ...   
1920/6300              1
19200/1000000          1
4340/10000             1
434/1000               1
3173/4000              1
Name: quota, Length: 12044, dtype: int64

### Descriptive Analysis

In [50]:
#CHECK THE STATISTICAL SUMMARY OF THE DATAFRAME
df.describe()

,Total_IMM,IMM_MILANO_HINTERLAND_No_small_cities,IMM_MILANO_HINTERLAND_Small_cities,IMM_ROMA_TERRITORIO,IMM_ROMA_PROVINCIA,IMM_TORINO_TERRITORIO,IMM_TORINO_PROVINCIA,IMM_FIRENZE_TERRITORIO,IMM_FIRENZE_PROVINCIA,IMM_GENOVA_TERRITORIO,IMM_GENOVA_PROVINCIA,IMM_OTHER,person_add_time,is_archived,pipeline_id,is_lead
count,3945057.00,3945057.00,3945057.00,3945057.00,3945057.00,3945057.00,3945057.00,3945057.00,3945057.00,3945057.00,3945057.00,3945057.00,227493,227683.00,227683.00,227683.00
mean,1.93,0.61,0.69,0.45,0.10,0.10,0.09,0.08,0.00,0.03,0.01,0.38,2023-11-22 08:08:28.117875456,0.58,0.21,0.89
min,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2022-06-11 06:15:36,0.00,0.00,0.00
25%,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2023-09-11 09:55:58,0.00,0.00,1.00
50%,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2024-01-11 22:37:44,0.00,0.00,1.00
75%,2.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2024-03-06 11:09:13,1.00,0.00,1.00
max,74692.00,42183.00,47011.00,74692.00,180.00,10000.00,148.00,7253.00,78.00,991.00,67.00,9816.00,2024-04-02 17:23:00,2.00,2.00,1.00
std,52.68,28.82,32.50,40.27,0.52,6.19,0.78,3.72,0.11,0.66,0.16,5.89,NaN,0.68,0.62,0.31


### Data Visualization

In [ ]:
#PLOT THE DISTRIBUTION OF THE DEAL VALUES
plt.figure(figsize=(10, 6))

In [ ]:
sns.histplot(df['deal_value'], bins=30, kde=True)
plt.title('Distribution of Deal Values')
plt.xlabel('Deal Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#PLOT THE DISTRIBUTION OF THE DEAL VALUES
plt.figure(figsize=(10, 6))

In [ ]:
sns.boxplot(df['deal_value'])
plt.title('Distribution of Deal Values')
plt.xlabel('Deal Value')
plt.show()

# Feature Engineering

In [ ]:
#CREATE A NEW COLUMN FOR THE DEAL VALUE
df['deal_value_log'] = np.log1p(df['deal_value'])

In [ ]:
#PLOT THE DISTRIBUT
plt.figure(figsize=(10, 6))

In [ ]:
sns.histplot(df['deal_value_log'], bins=30, kde=True)
plt.title('Distribution of Log Deal Values')
plt.xlabel('Log Deal Value')
plt.ylabel('Frequency')
plt.show()

# Feature Selection

In [ ]:
#SELECT THE COLUMNS FOR THE CLUSTERING
X = df[['deal_value_log']]

In [ ]:
#CHECK THE SHAPE OF THE DATAFRAME
X.shape

In [ ]:
#CHECK THE FIRST FEW ROWS OF THE DATAFRAME
X.head()

In [ ]:
#PLOT THE DISTRIBUTION OF THE DEAL VALUES
plt.figure(figsize=(10, 6))

In [ ]:
sns.histplot(X['deal_value_log'], bins=30, kde=True)
plt.title('Distribution of Log Deal Values')
plt.xlabel('Log Deal Value')
plt.ylabel('Frequency')
plt.show()

# Model Training

In [ ]:
#DEFINE THE NUMBER OF CLUSTERS
n_clusters = 3

In [ ]:
#CREATE THE KMEANS MODEL
kmeans = KMeans(n_clusters=n_clusters, random_state=0)

In [ ]:
#FIT THE MODEL
kmeans.fit(X)

In [ ]:
#PREDICT THE CLUSTERS
df['cluster'] = kmeans.predict(X)

In [ ]:
#PLOT THE CLUSTERS
plt.figure(figsize=(10, 6))

In [ ]:
sns.scatterplot(x='deal_value_log', y='deal_value', hue='cluster', data=df)
plt.title('Clusters of Deal Values')
plt.xlabel('Log Deal Value')
plt.ylabel('Deal Value')
plt.show()

# Transfer Learning

# Model Evaluation

# Deployment

# Monitoring